In [1]:
from bastionlab import Connection
import polars as pl

/home/kbamponsem/base/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = Connection("localhost").client

In [3]:
target_names = ["setosa", "versicolor", "virginica"]
df = pl.read_csv(
    "iris.csv",
    has_header=False,
    new_columns=[
        "sepal length (cm)",
        "sepal width (cm)",
        "petal length (cm)",
        "petal width (cm)",
        "target",
    ],
)

df = df.with_column(
    pl.col("target")
    .map(lambda a: a.apply(lambda b: 1 if target_names[b] == "virginica" else 0))
    .alias("target_class")
)


target_class_df = pl.DataFrame({"classes": target_names})
target_class_df = target_class_df.with_row_count("id")

In [4]:
rdf = client.polars.send_df(df)

classes_rdf = client.polars.send_df(target_class_df)

target_rdf = rdf.select(pl.col("target_class")).to_array()
rdf = rdf.select(["petal length (cm)", "petal width (cm)"]).collect()

test_rdf = rdf.tail(1).to_array()
train_rdf = rdf.to_array()

In [5]:
from bastionlab.linfa.trainers import LogisticRegression


lr = LogisticRegression()

lr.fit(train_rdf, target_rdf)

FittedModel(identifier=1cd3b51d-c23a-4e57-8b83-abd57c6d8596)
  └── LogisticRegression(alpha=1.0, gradient_tolerance=0.0001, fit_intercept=True, max_iterations=100, initial_params=[])

In [6]:
lr.predict(test_rdf.to_array()).fetch()

prediction
u64
1


In [7]:
# res = client.linfa.predict_proba(trainer, [5.5, 4.2])
# print(res)

In [8]:
from bastionlab.linfa import cross_validate

cross_validate(lr, train_rdf, target_rdf, cv=5, scoring="f1_score").fetch()

f1_score
f32
NaN
